In [2]:
import respy as rp
import numpy as np

from estimagic.optimization.optimize import maximize

params_corr, options_base, df = rp.get_example_model("kw_97_basic", True)

constr_base = rp.get_parameter_constraints("kw_97_basic")
constr_base[0].update({"bounds_distance": 1e-3})

optimization_options = dict()
optimization_options["algorithm"] = "nlopt_bobyqa"
optimization_options["algo_options"] = {"maxeval": 1}
optimization_options["constraints"] = constr_base

crit_func = rp.get_crit_func(params_corr, options_base, df)
maximize(crit_func, params_corr, **optimization_options)


({'status': 'success',
  'fitness': -59.032607867399385,
  'x': array([ 7.87000000e-01,  8.80430000e+00,  9.38000000e-02,  1.17000000e-01,
         -4.61000000e-02,  7.48000000e-02,  7.70000000e-03,  8.91560000e+00,
          1.89000000e-02,  6.74000000e-02,  1.42400000e-01, -1.77400000e-01,
          1.02100000e-01,  8.47040000e+00,  4.43000000e-02,  3.39100000e-01,
         -2.99000000e+00,  4.39480000e+04, -2.98300000e+03, -2.63570000e+04,
          1.68870000e+04,  3.30100000e-01, -1.26701740e-01,  3.07845869e-01,
         -1.21999040e-01,  9.71697859e-02,  2.91724025e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  2.31200000e+03,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.33940000e+04,
          4.13300000e-01,  3.08900000e-01,  1.25900000e-01, -6.68000000e-02,
          2.99600000e-01, -2.63520000e+04,  2.15000000e+02, -4.22100000e-01,
         -1.22300000e-01, -3.05410000e+04, -1.69660000e+04, -4.99800000e-01,
          7.56